In [387]:
import pandas as pd

from datetime import datetime,timedelta

In [388]:
df = pd.read_csv('big mac.csv')

In [389]:
df

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price,USD_raw,EUR_raw,GBP_raw,JPY_raw,CNY_raw,GDP_dollar,adj_price,USD_adjusted,EUR_adjusted,GBP_adjusted,JPY_adjusted,CNY_adjusted
0,2000-04-01,ARG,ARS,Argentina,2.50,1.0000,2.500000,-0.00398,0.05007,-0.16722,-0.09864,1.09091,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-04-01,AUS,AUD,Australia,2.59,1.6800,1.541667,-0.38579,-0.35246,-0.48645,-0.44416,0.28939,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-04-01,BRA,BRL,Brazil,2.95,1.7900,1.648045,-0.34341,-0.30778,-0.45102,-0.40581,0.37836,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-04-01,CAN,CAD,Canada,2.85,1.4700,1.938776,-0.22758,-0.18566,-0.35417,-0.30099,0.62152,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-04-01,CHE,CHF,Switzerland,5.90,1.7000,3.470588,0.38270,0.45774,0.15609,0.25130,1.90267,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1381,2020-07-01,UKR,UAH,Ukraine,59.00,27.1300,2.174714,-0.61914,-0.54562,-0.49157,-0.40181,-0.29813,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1382,2020-07-01,URY,UYU,Uruguay,189.00,43.6750,4.327418,-0.24213,-0.09584,0.01171,0.19032,0.39664,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1383,2020-07-01,USA,USD,United States,5.71,1.0000,5.710000,0.00000,0.19303,0.33494,0.57062,0.84286,62868.917,5.174502,0.000,-0.019,0.122,0.279,0.070
1384,2020-07-01,VNM,VND,Vietnam,66000.00,23180.0000,2.847282,-0.50135,-0.40510,-0.33433,-0.21681,-0.08106,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [390]:
data_frames = {}
countries = set(df["name"].tolist())
print(len(countries))
to_drop = ["USD_raw", "EUR_raw", "GBP_raw", "JPY_raw", "CNY_raw", "GDP_dollar", "adj_price", "USD_adjusted", "EUR_adjusted", "GBP_adjusted", "JPY_adjusted", "CNY_adjusted", ]

57


In [391]:
def change_date_to_datetime(str):
    datetime_object = datetime.strptime(str, '%Y-%m-%d').date()
    return datetime_object


In [392]:
#change str to datetime object

df["date"] = df["date"].apply(change_date_to_datetime)
columns_tochange = ["local_price", "dollar_ex", "dollar_price"]

In [393]:
df = df.drop(to_drop, axis=1)

In [394]:
for country in countries:
    data_frames[country] = df[df["name"] == country]

Seperating Data frames by Country

In [395]:
year1999 = datetime(1999, 12, 31).date()
year2021 = datetime(2021, 1, 1).date()
x = data_frames["Canada"]["date"].iloc[0]

In [396]:
def nearest(items, pivot):
    return min(items, key=lambda z: abs(z - pivot))

In [397]:
years = []
for i in range(2000, 2021):
    years.append(datetime(i, 1, 1).date())
for country in countries:
    # Enure all data is is in range 2000 - 2020
    frame = data_frames[country]
    data_frames[country] = frame[((frame["date"] > year1999) &  (year2021 > frame["date"]))]
    for year in years:
        x = frame[((frame["date"] >= year) &  ((year+pd.DateOffset(years=1)) > frame["date"]))]
        if len(x) > 1:
            y = x["date"].tolist()
            midlest = nearest(y, (year+timedelta(days=150)))
            to_delete = x[((x["date"] > midlest) |  (midlest > x["date"]))]
            data_frames[country] = pd.concat([data_frames[country], to_delete])
    data_frames[country] = data_frames[country].drop_duplicates(keep=False)
        

In [398]:
final_list = []
for country in countries.copy():
    if len(data_frames[country]) < 21:
        countries.remove(country)
        del data_frames[country]
    else:
        final_list.append(data_frames[country])
        
final_df = pd.concat(final_list)

In [399]:
final_df = final_df.reset_index(drop=True)
final_df

,date,iso_a3,currency_code,name,local_price,dollar_ex,dollar_price
0,2000-04-01,CHL,CLP,Chile,1260.0,514.00,2.451362
1,2001-04-01,CHL,CLP,Chile,1260.0,601.00,2.096506
2,2002-04-01,CHL,CLP,Chile,1400.0,655.00,2.137405
3,2003-04-01,CHL,CLP,Chile,1400.0,716.00,1.955307
4,2004-05-01,CHL,CLP,Chile,1400.0,641.00,2.184087
...,...,...,...,...,...,...,...
562,2016-07-01,KOR,KRW,South Korea,4400.0,1140.95,3.856435
563,2017-07-01,KOR,KRW,South Korea,4400.0,1144.65,3.843970
564,2018-07-01,KOR,KRW,South Korea,4500.0,1116.00,4.032258
565,2019-07-09,KOR,KRW,South Korea,4500.0,1180.55,3.811783


In [400]:
final_df.to_csv("cleaned_bigmac.csv", index=False)